In [1]:
# notebook.ipynb
# Cell 1: Imports and Setup
import os
import sys
import yaml
import importlib
import time
import shutil
from tool_registry import ToolRegistry
from langchain_core.tools import StructuredTool

# Set working directory to project root (adjust path if notebook is elsewhere)
project_root = os.path.abspath(os.path.dirname(os.getcwd()))  # Assumes notebook is in project_root/
os.chdir(project_root)
sys.path.insert(0, project_root)  # Ensure project_root is in sys.path
print(f"Working directory: {os.getcwd()}")
print(f"sys.path: {sys.path}")


Working directory: /media/dinith/Academic/Next/gai/test1
sys.path: ['/media/dinith/Academic/Next/gai/test1', '/usr/lib/python313.zip', '/usr/lib/python3.13', '/usr/lib/python3.13/lib-dynload', '', '/media/dinith/Academic/.venv/lib/python3.13/site-packages']


In [2]:

# Cell 2: ToolRegistry Definition (if not in tool_registry.py)
# Skip this cell if tool_registry.py exists
class ToolRegistry:
    _tools: dict = {}

    @classmethod
    def register(cls, func) -> callable:
        """Register a function or StructuredTool as a tool."""
        if isinstance(func, StructuredTool):
            tool_name = func.name
        else:
            tool_name = func.__name__
        cls._tools[tool_name] = func
        print(f"Registered tool: {tool_name}")
        return func

    @classmethod
    def get_tools(cls) -> list:
        """Return all registered tools."""
        return list(cls._tools.values())

    @classmethod
    def get_tool(cls, name: str) -> callable:
        """Get a tool by name."""
        return cls._tools.get(name)

In [3]:
# Cell 3: Write Tool File (if generating programmatically)
def write_tool_file():
    """Write tools/math_tools.py programmatically."""
    code = """
from tool_registry import ToolRegistry
from langchain_core.tools import tool

@ToolRegistry.register
@tool
def subtract(input: str) -> float:
    \"""Subtract two numbers from string 'a,b'.\"""
    a, b = map(float, input.split(","))
    return a - b

@ToolRegistry.register
@tool
def addition(input: str) -> float:
    \"""Add two numbers from string 'a,b'.\"""
    a, b = map(float, input.split(","))
    return a + b
"""
    os.makedirs("tools", exist_ok=True)
    with open("tools/math_tools.py", "w") as f:
        f.write(code)
        f.flush()
        os.fsync(f.fileno())
    time.sleep(0.2)  # Increased delay for notebook environment
    if not os.path.exists("tools/math_tools.py"):
        raise FileNotFoundError("Failed to create tools/math_tools.py")


In [4]:



# Cell 4: Clear Cache and Load Tools
def clear_pycache():
    """Clear stale .pyc files."""
    if os.path.exists("tools/__pycache__"):
        shutil.rmtree("tools/__pycache__")

def load_tools_from_yaml(config_path: str) -> list:
    """Load tools from YAML config using ToolRegistry."""
    clear_pycache()  # Clear stale .pyc files
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file {config_path} not found")
    time.sleep(0.2)  # Increased delay for notebook

    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)

    if not config or 'tools' not in config:
        raise ValueError("Invalid YAML config: 'tools' key missing")

    tools = []
    for tool_config in config['tools']:
        module_name = tool_config.get('module')
        try:
            if module_name in sys.modules:
                importlib.reload(sys.modules[module_name])
            importlib.import_module(module_name)
        except ImportError as e:
            print(f"Error importing module {module_name}: {e}")
            continue

        tool_name = tool_config.get('name')
        func = ToolRegistry.get_tool(tool_name)
        if func is None:
            print(f"Tool '{tool_name}' not found in registry")
            continue
        tools.append(func)

    return tools


In [5]:
write_tool_file()  # Generate tools/math_tools.py

In [6]:

# Cell 5: Run the Workflow

tools = load_tools_from_yaml("/media/dinith/Academic/Next/gai/test1/way2/tools.yaml")
print(f"Loaded tools: {[t.name if hasattr(t, 'name') else t.__name__ for t in tools]}")

Tool 'subtract' not found in registry
Tool 'addition' not found in registry
Loaded tools: []


FileNotFoundError: Config file config.yaml not found

In [ ]:
# # main.py
# import os
# import sys
# import yaml
# import importlib
# import time
# import shutil
# from tool_registry import ToolRegistry

# # Add project root to sys.path
# project_root = os.path.abspath(os.path.dirname(__file__))
# sys.path.insert(0, project_root)

# def write_tool_file():
#     """Write tools/math_tools.py programmatically."""
#     code = """
# from tool_registry import ToolRegistry
# from langchain_core.tools import tool

# @ToolRegistry.register
# @tool
# def subtract(input: str) -> float:
#     \"""Subtract two numbers from string 'a,b'.\"""
#     a, b = map(float, input.split(","))
#     return a - b

# @ToolRegistry.register
# @tool
# def addition(input: str) -> float:
#     \"""Add two numbers from string 'a,b'.\"""
#     a, b = map(float, input.split(","))
#     return a + b
# """
#     os.makedirs("tools", exist_ok=True)
#     with open("tools/math_tools.py", "w") as f:
#         f.write(code)
#         f.flush()
#         os.fsync(f.fileno())
#     time.sleep(0.1)  # Ensure file system settles

# def clear_pycache():
#     """Clear stale .pyc files."""
#     if os.path.exists("tools/__pycache__"):
#         shutil.rmtree("tools/__pycache__")

# def load_tools_from_yaml(config_path: str) -> list:
#     """Load tools from YAML config using ToolRegistry."""
#     clear_pycache()  # Clear stale .pyc files
#     if not os.path.exists(config_path):
#         raise FileNotFoundError(f"Config file {config_path} not found")
#     time.sleep(0.1)  # File system delay

#     with open(config_path, 'r') as f:
#         config = yaml.safe_load(f)

#     if not config or 'tools' not in config:
#         raise ValueError("Invalid YAML config: 'tools' key missing")

#     tools = []
#     for tool_config in config['tools']:
#         module_name = tool_config.get('module')
#         try:
#             if module_name in sys.modules:
#                 importlib.reload(sys.modules[module_name])
#             importlib.import_module(module_name)
#         except ImportError as e:
#             print(f"Error importing module {module_name}: {e}")
#             continue

#         tool_name = tool_config.get('name')
#         func = ToolRegistry.get_tool(tool_name)
#         if func is None:
#             print(f"Tool '{tool_name}' not found in registry")
#             continue
#         tools.append(func)

#     return tools

# # Example usage
# if __name__ == "__main__":
#     # write_tool_file()  # Generate tools/math_tools.py
#     tools = load_tools_from_yaml("config.yaml")
#     print(f"Loaded tools: {[t.name if hasattr(t, 'name') else t.__name__ for t in tools]}")

NameError: name '__file__' is not defined